# Agora vamos falar de aprendizado supervisionado, usando um algoritmo clássico: Support Vector Machine (SVM).

In [0]:
#Importando bibliotecas básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<h3> Segue o link para visualizar os dados: 
    
https://github.com/Natalnet/GCiD/raw/master/Codes/Data/INMET-Dados_Diarios_PortoAlegre_1980-2017.csv

In [0]:
#Lendo os dados
dataSet = pd.read_csv("https://github.com/Natalnet/GCiD/raw/master/Codes/Data/INMET-Dados_Diarios_PortoAlegre_1980-2017.csv",
                      skiprows = 16, sep=';')
#Transformando data em variável do tipo datetime
dataSet["Data"] = pd.to_datetime(dataSet["Data"], format = "%d/%m/%Y")

In [3]:
#Visualizando cabeçalho dos dados
dataSet.head()

,Estacao,Data,Hora,Precipitacao,TempMaxima,TempMinima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Unnamed: 11
0,83967,1980-01-01,0,NaN,23.0,NaN,9.8,5.5,18.34,65.0,3.800000,NaN
1,83967,1980-01-01,1200,0.0,NaN,14.8,NaN,NaN,NaN,NaN,NaN,NaN
2,83967,1980-01-02,0,NaN,24.1,NaN,10.8,4.8,18.90,69.5,2.200000,NaN
3,83967,1980-01-02,1200,0.0,NaN,13.8,NaN,NaN,NaN,NaN,NaN,NaN
4,83967,1980-01-03,0,NaN,28.4,NaN,11.7,5.0,21.24,59.0,2.166667,NaN


In [4]:
#Visualizando descrição resumida dos dados
dataSet.describe()

,Estacao,Hora,Precipitacao,TempMaxima,TempMinima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Unnamed: 11
count,24542.0,24542.000000,12266.000000,12242.000000,12243.000000,12131.000000,11600.000000,12227.000000,12223.000000,12269.000000,0.0
mean,83967.0,600.000000,3.959351,25.398268,15.728727,5.888220,2.432095,19.738193,76.451771,2.071611,NaN
std,0.0,600.012224,10.226288,5.637669,4.861767,3.946654,1.541723,4.861474,10.047764,1.293506,NaN
min,83967.0,0.000000,0.000000,8.600000,-0.200000,0.000000,0.000000,5.160000,37.500000,0.000000,NaN
25%,83967.0,0.000000,0.000000,21.300000,12.500000,2.100000,1.300000,16.280000,69.500000,1.200000,NaN
50%,83967.0,600.000000,0.000000,25.800000,16.300000,6.700000,2.200000,20.220000,76.500000,1.866667,NaN
75%,83967.0,1200.000000,1.800000,29.600000,19.600000,9.100000,3.200000,23.560000,83.750000,2.700000,NaN
max,83967.0,1200.000000,149.600000,40.600000,27.900000,13.200000,20.700000,33.700000,99.250000,14.500000,NaN


# Esse problema encontrado agora na base de dados já foi enfrentado anteriormente, portanto, vamos agilizar o processo com a função "OrganizarDados"

In [0]:
def OrganizarDados(dataSet):
    #Transformando data em variável do tipo datetime
    dataSet["Data"] = pd.to_datetime(dataSet["Data"], format = "%d/%m/%Y")
    #Atribuindo "Data" como índice para a base de dados
    dataSet = dataSet.set_index("Data")
    #Separando os dados em dois dataframes, um para as 00:00 h e outro para as 12:00 h
    dataSet00, dataSet12 = dataSet[dataSet["Hora"] == 0], dataSet[dataSet["Hora"] == 1200]
    #Descartando as colunas "Hora", "Estacao" e "Unnamed: 11" 
    dataSet00, dataSet12 = dataSet00.drop(columns = ["Hora", "Estacao", "Unnamed: 11"]), dataSet12.drop(columns = ["Hora", "Estacao", "Unnamed: 11"])
    #Eliminando colunas completas por "NaN" em cada uma das bases de dados
    dataSet00 = dataSet00.drop(columns = ["Precipitacao","TempMinima"])
    dataSet12 = dataSet12.drop(columns = ["TempMaxima","Insolacao","Evaporacao Piche","Temp Comp Media","Umidade Relativa Media","Velocidade do Vento Media"])
    #Criando o intervalo completo de tempo de 01-01-1980 à 31-12-2017
    dataInicial = '2005-01-01'
    dataFinal = '2017-12-31'
    tempo = pd.date_range(dataInicial, dataFinal)
    #Atribuindo este intervalo de tempo à um dataSet provisório
    dataSetProv = pd.DataFrame()
    dataSetProv["Data"] = tempo
    #Atribuindo o índice para o dataSet provisório como sendo a coluna de datas
    dataSetProv = dataSetProv.set_index("Data")
    #Mesclando o dataSet provisório como sendo o resultado da junção dos dataSet00 e dataSet12
    dataSetProv = dataSetProv.join(dataSet00).join(dataSet12)
    #Tornando o dataSet como sendo o dataSet provisório
    dataSet = dataSetProv  
    return dataSet

In [0]:
#Aplicando a função ao dataSet
dataSet = OrganizarDados(dataSet)

In [0]:
#Tornando a informação de mês uma variável do dataSet
dataSet["Mes"] = dataSet.index.month

In [0]:
#Eliminando eventos inconsistentes da base de dados
dataSet = dataSet.dropna()

In [0]:
#Resentando o índice do dataSet
dataSet = dataSet.reset_index(drop = True)

In [10]:
#Visualizando cabeçalho dos dados
dataSet.head()

,TempMaxima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Precipitacao,TempMinima,Mes
0,30.8,9.4,3.2,25.28,70.50,2.866667,0.0,21.0,1
1,34.5,7.7,2.8,26.84,71.00,2.733333,0.0,21.3,1
2,27.0,0.1,3.7,24.36,88.75,0.966667,3.5,23.4,1
3,32.6,7.9,1.0,26.48,72.50,2.200000,3.2,22.3,1
4,31.5,9.3,3.0,26.04,76.00,2.466667,0.0,22.0,1


In [11]:
#Visualizando descrição resumida dos dados
dataSet.describe()

,TempMaxima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Precipitacao,TempMinima,Mes
count,4742.000000,4742.000000,4742.000000,4742.000000,4742.000000,4742.000000,4742.000000,4742.000000,4742.000000
mean,25.719338,5.952952,2.100084,19.977912,77.014287,2.064012,4.189350,16.010924,6.520877
std,5.670541,3.961481,1.225170,4.864214,9.922657,1.081291,10.976284,4.776053,3.449073
min,8.600000,0.000000,0.000000,5.360000,37.500000,0.000000,0.000000,0.100000,1.000000
25%,21.500000,2.200000,1.200000,16.560000,70.000000,1.333333,0.000000,12.900000,4.000000
50%,26.200000,6.700000,2.000000,20.340000,77.000000,1.916667,0.000000,16.500000,7.000000
75%,30.000000,9.100000,2.800000,23.760000,84.250000,2.633333,2.100000,19.700000,10.000000
max,40.600000,13.000000,11.900000,33.700000,98.250000,9.200000,149.600000,27.900000,12.000000


In [0]:
#Separando a coluna "Mes" do DataFrame
mes = dataSet["Mes"].values
dataSet = dataSet.drop(columns = "Mes")

In [0]:
from sklearn.model_selection import train_test_split
#Separando os conjuntos de treino e teste com os dados embaralhados, sendo 90% para treino e 10% para teste
X_train, X_test, y_train, y_test = train_test_split(dataSet, mes, test_size = 0.2, random_state = 9)

In [0]:
from sklearn.preprocessing import StandardScaler
#Efetuando uma transformação no conjunto de dados - também já estudamos isso
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

<h3> Vamos agora utilizar o algoritmo SVM para Classificação, disponibilizado na biblioteca Scikit-Learn. Segue link para a documentação:

https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC

In [0]:
from sklearn.svm import SVC
#Criação do objeto SVC como "classifier"
classifier = SVC(C = 10000000.0, kernel = 'rbf', random_state = 0, gamma = 0.0001)

In [16]:
#Treinando o modelo classificador
classifier.fit(X_train, y_train)

SVC(C=10000000.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [0]:
#Previsão para os valores de teste
y_pred = classifier.predict(X_test)

In [0]:
from sklearn.metrics import confusion_matrix
#Avaliando o modelo com matriz de confusão
cm = confusion_matrix(y_test, y_pred)
#Convertendo a matriz de confusão para um DataFrame, para facilitar a visualização
cm = pd.DataFrame(cm)
#Declarando títulos para as colunas e os índices da matriz de confusão, para facilitar a interpretação
cm.columns = ["Jan","Fev","Mar","Abr","Mai","Jun","Jul","Ago","Set","Out","Nov","Dez"]
cm.index = ["Jan","Fev","Mar","Abr","Mai","Jun","Jul","Ago","Set","Out","Nov","Dez"]

In [19]:
#Imprimindo a matriz de confusão
cm

,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez
Jan,26,14,11,3,0,0,1,0,0,1,8,15
Fev,26,23,17,4,0,0,0,0,0,0,2,6
Mar,13,9,34,7,2,0,0,0,1,8,9,8
Abr,1,1,13,14,11,1,2,2,9,11,4,4
Mai,0,0,0,8,30,14,10,2,10,1,0,0
Jun,0,0,1,1,12,36,16,6,9,1,0,0
Jul,0,0,0,4,17,17,26,8,7,2,0,0
Ago,0,0,0,4,20,8,17,25,12,4,1,0
Set,0,0,1,5,16,3,2,6,21,16,4,0
Out,1,1,4,15,8,0,1,2,14,25,8,2
